In [1]:
import pandas as pd

In [2]:
import orca

In [3]:
import os; os.chdir('../')

In [4]:
from scripts import datasources

In [5]:
trips = pd.read_csv('./data/persons_HWtrips.csv')

In [6]:
trips.head()

,person_id,member_id,age,primary_commute_mode,relate,edu,sex,hours,hispanic,earning,...,work_at_home,worker,household_id,node_id_small,node_id_walk,job_id,MODE,HW_departure_time,WH_departure_time,dwell
0,0,1,47,1,0,22,1,40,1,85000,...,0,1,0,6.546892e+07,5.125306e+09,413751.0,drive,10.254347,13.220331,2.965985
1,2,1,47,1,0,22,1,40,1,85000,...,0,1,740,2.279076e+09,2.860970e+08,1258351.0,drive,7.080983,8.064803,0.983820
2,3,1,47,1,0,22,1,40,1,85000,...,0,1,975,3.969451e+09,6.546673e+07,343693.0,drive,8.481918,11.704230,3.222312
3,4,1,47,1,0,22,1,40,1,85000,...,0,1,977,6.561551e+07,6.554853e+07,447704.0,drive,8.544929,17.337526,8.792596
4,5,1,47,1,0,22,1,40,1,85000,...,0,1,2321,3.531401e+08,6.555442e+07,193894.0,drive,9.975130,15.818964,5.843834


In [6]:
job_coords = orca.merge_tables('jobs', ['jobs', 'buildings', 'parcels'])
job_coords = job_coords[['x', 'y']]

/home/max/anaconda3/envs/ual-model/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/max/projects/ual_model_workspace/fall-2018-models/scripts/datasources.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['res_sqft_per_unit'][df['res_sqft_per_unit'] == np.inf] = 0


In [7]:
hh_coords = orca.merge_tables('households', ['households', 'units', 'buildings', 'parcels'])
hh_coords = hh_coords[['x', 'y']]

/home/max/anaconda3/envs/ual-model/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
trips = trips[['person_id', 'household_id', 'job_id', 'MODE', 'HW_departure_time', 'WH_departure_time']].rename(
    columns={'HW_departure_time': 'Home', 'WH_departure_time': 'Work'})

In [9]:
trip_data = trips.merge(
    hh_coords, left_on='household_id', right_index=True).merge(
    job_coords, left_on='job_id', right_index=True, suffixes=('_home', '_work'))
trip_data = trip_data[['person_id', 'MODE', 'Home', 'Work', 'x_home', 'y_home', 'x_work', 'y_work']]

In [10]:
melted = trip_data.melt(
    id_vars=['person_id', 'MODE', 'x_home', 'y_home', 'x_work', 'y_work'],
    var_name='activityType', value_name='endTime')

In [11]:
melted['x'] = None
melted['y'] = None
melted.loc[melted['activityType'] == 'Home', 'x'] = melted.loc[melted['activityType'] == 'Home', 'x_home']
melted.loc[melted['activityType'] == 'Home', 'y'] = melted.loc[melted['activityType'] == 'Home', 'y_home']
melted.loc[melted['activityType'] == 'Work', 'x'] = melted.loc[melted['activityType'] == 'Work', 'x_work']
melted.loc[melted['activityType'] == 'Work', 'y'] = melted.loc[melted['activityType'] == 'Work', 'y_work']

In [13]:
plans = melted.sort_values(['person_id', 'endTime'])[['person_id', 'MODE', 'activityType', 'endTime', 'x', 'y']].reset_index(drop=True)

In [21]:
plans['planElement'] = 'activity'
plans['planElementIndex'] = plans.groupby('person_id').cumcount() * 2 + 1

In [23]:
returnActivity = plans[plans['planElementIndex'] == 1]

In [24]:
returnActivity.loc[:, 'planElementIndex'] = 5
returnActivity.loc[:, 'endTime'] = ''

/home/max/anaconda3/envs/ual-model/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [25]:
plans = plans.append(returnActivity, ignore_index=True).sort_values(['person_id', 'planElementIndex'])

In [26]:
legs = plans[plans['planElementIndex'].isin([1,3])]

In [27]:
legs.loc[:, 'planElementIndex'] = legs.loc[:, 'planElementIndex'] + 1

/home/max/anaconda3/envs/ual-model/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [28]:
legs.loc[:, 'activityType'] = ''
legs.loc[:, 'endTime'] = ''
legs.loc[:, 'x'] = ''
legs.loc[:, 'y'] = ''
legs.loc[:, 'planElement'] = 'leg'

/home/max/anaconda3/envs/ual-model/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [29]:
plans = plans.append(legs, ignore_index=True).sort_values(['person_id', 'planElementIndex']).rename(
    columns={'person_id': 'personId', 'MODE': 'mode'}).reset_index(drop=True)
plans = plans[['personId', 'planElement', 'planElementIndex', 'activityType', 'x', 'y', 'endTime', 'mode']]

In [35]:
plans.loc[plans['planElement'] == 'activity', 'mode'] = ''

In [38]:
plans.to_csv('./data/urbansim_beam_plans.csv', index=False)